$$
    S_t = S_0 e^{\left(r - \frac{1}{2}\sigma^2\right)t + \sigma W_t}
$$

In [3]:
import numpy as np
from numpy.random import randn, seed
from numpy import exp
import matplotlib.pyplot as plt

In [ ]:
S0 = 100
r = 0.06
sigma = 0.23
W_t =  
S_t = S0 * exp(r - sigma ** 2 / 2) * 